In [3]:
import cv2
import numpy as np
import gmpy2 
import math
from random import randint
from math import gcd
import libnum
from PIL import Image
import timeit
rs = gmpy2.random_state()


In [4]:
### read the .mp4 file and save it as np.array type
### 380 296
cap = cv2.VideoCapture('video1.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

array_o = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

num = 0
ret = True
fps = 10

while (num < frameCount  and ret):
    ret, array_o[num] = cap.read()
    num += 1

In [5]:
### paillier method
class Paillier(object):
    
 
    def __init__(self):
        
        self.publicKey = None
        self.privateKey = None
        
    def lcm(self, x, y):
        
        return x * y // gcd(x, y) 
    
    def sTn(self,m):
        arr = bytes(m, 'utf-8')
        m = int.from_bytes(arr, 'big')
        return m
          
     
    def getP(self):
        
        p = gmpy2.mpz_urandomb(rs, 1024)
        while not gmpy2.is_prime(p):
            p = p + 1
        return p
    
    def getKeys(self):
     
        #p = self.getP()
        #q = self.getP()
        
        p = 23
        q = 19
        
        n = p * q
        lamda = self.lcm(p-1, q-1)
        
        
        g = n + 1
        mu = (pow(g,lamda,n*n)-1) // n
        mu = libnum.invmod(mu, n)
        
        #mu = gmpy2.invert(lamda, n)
        
        n = int(n)
        g = int(g)
        lamda = int(lamda)
        mu = int(mu)
              
        self.publicKey = [n, g]
        self.privateKey = [lamda, mu]
    
    def encrypt(self, m):
        
        if isinstance(m, str):
            m = self.sTn(m)
        
        #print(m)
        
        n,g = self.publicKey
        r = randint(0,n)
        while gcd(n, r)!= 1:
            r = r + 1
        
        c = (pow(g, m, n*n) * pow(r, n, n*n)) % (n*n)
        
        # print(c)
        return c
    
    def decrypt(self,c):
        
        n, g = self.publicKey
        lamda, mu = self.privateKey
        
        c = (pow(c,lamda, n*n) - 1) // n * mu
        c = c % n
        c = int (c)
        # print(c)
        m = c
        #m = libnum.n2s(c)
        #m = m.decode("utf-8") 
        return m

In [6]:
### generate keys
p = Paillier()
p.getKeys()

### decode the array for the video
start = timeit.default_timer()

data_encrypted = [[[[p.encrypt(int(x)) for x in row] for row in mat]for mat in frame]for frame in array_o] 

stop = timeit.default_timer()

print('Time for encoding: ', stop - start, 's') 

Time for encoding:  104.9053673 s


In [7]:
##### create video with encoded array
fourcc = cv2.VideoWriter_fourcc(*'MP42') 

array_code = np.array(data_encrypted, dtype=np.uint8)
video = cv2.VideoWriter('encode v1.mp4', fourcc, float(fps), (frameWidth, frameHeight))

for count in range(frameCount):
    img = array_code[count]
    video.write(img)

video.release()

In [8]:
#### decode array
start = timeit.default_timer()

data_decrypted = [[[[p.decrypt(x) for x in row] for row in mat]for mat in frame ]for frame in data_encrypted]

stop = timeit.default_timer()

print('Time for decoding: ', stop - start, 's') 

Time for decoding:  44.63018630000002 s


In [9]:
### create video with decoded array
array_decode = np.array(data_decrypted, dtype=np.uint8)

video = cv2.VideoWriter('decode v1.mp4', fourcc, float(fps), (frameWidth, frameHeight))
for count in range(frameCount):
    img = array_decode[count]
    video.write(img)

video.release()

In [13]:
### read the .mp4 file and save it as np.array type
### 720 720
cap = cv2.VideoCapture('video2.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

array_o = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

num = 0
ret = True
fps = 12

while (num < frameCount  and ret):
    ret, array_o[num] = cap.read()
    num += 1

In [14]:
### generate keys
p = Paillier()
p.getKeys()

### decode the array for the video
start = timeit.default_timer()

data_encrypted = [[[[p.encrypt(int(x)) for x in row] for row in mat]for mat in frame]for frame in array_o] 

stop = timeit.default_timer()

print('Time for encoding: ', stop - start, 's') 

#### decode array
start = timeit.default_timer()

data_decrypted = [[[[p.decrypt(x) for x in row] for row in mat]for mat in frame ]for frame in data_encrypted]

stop = timeit.default_timer()

print('Time for decoding: ', stop - start, 's') 

array_decode = np.array(data_decrypted, dtype=np.uint8)

video = cv2.VideoWriter('decode v2.mp4', fourcc, float(fps), (frameWidth, frameHeight))
for count in range(frameCount):
    img = array_decode[count]
    video.write(img)

video.release()

Time for encoding:  713.6603002000002 s
Time for decoding:  326.9919525999999 s
